In [27]:
import pandas as pd
import numpy as np
from ast import literal_eval
import scipy.sparse

from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k, reciprocal_rank, auc_score
# To reimport:
# import tools.lightfm_tools
from tools.lightfm_tools import prepare_interactions, evaluate_model, evaluate_diversity_and_coverage, random_search
# import tools.lightfm_tools
# import importlib
# importlib.reload(tools.lightfm_tools)
# from tools.lightfm_tools import prepare_interactions, evaluate_model, evaluate_diversity_and_coverage, random_search


## Data loading

In [28]:
# Ratings without the least active users and the least popular games
df = pd.read_csv('data/ratings.csv.gz')
games_df = pd.read_json('./data/bgg_GameItem.jl', lines = True)
# Names of the most popular features
features_names = pd.read_csv('data/LightFM_item_features_names.csv.gz')
# Features of games
game_features = pd.read_csv('data/LightFM_item_features.csv.gz')
game_features['features'] = game_features.features.apply(literal_eval)

In [29]:
# We don't need features of games we don't have any interactions with
features_list = [x for x in features_names['0']]
games_list = [x for x in df['bgg_id']]
game_features = game_features.drop((game_features['bgg_id'])[~game_features['bgg_id'].isin(games_list)].index)

In [30]:
# Positive interactions are: rating the game above 6 or having a game in collection without rating it
df_positive = df.loc[((df['bgg_user_owned'] == 1.0) & (df['bgg_user_rating'].isnull()) | (df['bgg_user_rating'] > 6))].copy()
df_positive['value'] = 1

In [31]:
df_negative =  df.loc[(df['bgg_user_rating'] < 4)].copy()
df_negative['value'] = -1
df_logistic = pd.concat([df_positive, df_negative])

In [32]:
best_epochs_so_far = 49
best_params_so_far = {'no_components': 71, 'learning_schedule': 'adadelta', 'learning_rate': 0.05652168855133591, 'item_alpha': 2.816672985617472e-06, 'max_sampled': 9, 'rho': 0.9369181237831744, 'epsilon': 2.0854895195545605e-06}

## Preparation of dataset, interactions and item features matrices

In [33]:
dataset = Dataset()
dataset.fit((x for x in df_positive['bgg_user_name']), (x for x in df_positive['bgg_id']), item_features=(x for x in features_list))

In [34]:
# Interactions for all functions except logistic loss
try:
    interactions = scipy.sparse.load_npz('data/interactions.npz')
    test_interactions = scipy.sparse.load_npz('data/test_interactions.npz')
    known = pd.read_csv('data/known.csv.gz')
    print('Loaded interactions')
except:    
    interactions, test_interactions, known = prepare_interactions(df_positive, dataset)
    scipy.sparse.save_npz('data/interactions.npz', interactions)
    scipy.sparse.save_npz('data/test_interactions.npz', test_interactions)
    known.to_csv('data/known.csv.gz', compression='gzip', index=False)    

Loaded interactions


In [35]:
dataset_logistic = Dataset()
dataset_logistic.fit((x for x in df_logistic['bgg_user_name']), (x for x in df_logistic['bgg_id']), item_features=(x for x in features_list))

In [12]:
# Interactions for logistic loss
try:
    interactions_logistic = scipy.sparse.load_npz('data/interactions_logistic.npz')
    test_interactions_logistic = scipy.sparse.load_npz('data/test_interactions_logistic.npz')
    known_logistic = pd.read_csv('data/known_logistic.csv.gz')
    print("Loaded interactions")
except:    
    interactions_logistic, test_interactions_logistic, known_logistic = prepare_interactions(df_logistic, dataset)
    scipy.sparse.save_npz('data/interactions_logistic.npz', interactions_logistic)
    scipy.sparse.save_npz('data/test_interactions_logistic.npz', test_interactions_logistic)
    known_logistic.to_csv('data/known_logistic.csv.gz', compression='gzip', index=False)    

Loaded interactions


In [13]:
item_features = dataset.build_item_features((val['bgg_id'], [w for w in val['features'] if w in features_list]) for idx, val in game_features.iterrows())

In [37]:
id_item_features = dataset.build_item_features((val['bgg_id'], [w for w in val['features'] if w in features_list] + [val['bgg_id']]) for idx, val in game_features.iterrows())

In [ ]:
id_item_features_logistic = dataset_logistic.build_item_features((val['bgg_id'], [w for w in val['features'] if w in features_list] + [val['bgg_id']]) for idx, val in game_features.iterrows())

In [15]:
random_game_features = game_features.copy()
random_game_features.iloc[:, 1] = np.random.permutation(game_features.iloc[:,1].values)
random_item_features = dataset.build_item_features((val['bgg_id'], [w for w in val['features'] if w in features_list]) for idx, val in random_game_features.iterrows())

## Comparison of results achieved using different parameters

### WARP model

In [10]:
model = LightFM(loss='warp')
model.fit(interactions, verbose=True, epochs=20, num_threads=8)

Epoch: 100%|██████████| 20/20 [02:20<00:00,  7.02s/it]


In [17]:
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8)

Precision: train 0.50
Precision: test 0.20
AUC: train 0.94
AUC: test 0.93


In [18]:
features_model = LightFM(loss='warp')
features_model.fit(interactions, verbose=True, item_features = item_features,
                   epochs=20, num_threads=8)

Epoch: 100%|██████████| 20/20 [10:27<00:00, 31.35s/it]


In [19]:
evaluate_model(features_model, interactions, test_interactions, k=5,
               num_threads=8, item_features = item_features)

Precision: train 0.49
Precision: test 0.19
AUC: train 0.93
AUC: test 0.92


In [20]:
random_features_model = LightFM(loss='warp')
random_features_model.fit(interactions, verbose=True, item_features = random_item_features,
                          epochs=20, num_threads=8)

Epoch: 100%|██████████| 20/20 [09:29<00:00, 28.45s/it]


In [21]:
evaluate_model(random_features_model, interactions, test_interactions, k=5,
               num_threads=8, item_features = random_item_features)

Precision: train 0.48
Precision: test 0.18
AUC: train 0.93
AUC: test 0.92


#### WARP hyperparameters search

In [8]:
random_search(interactions, test_interactions, id_item_features, num_samples=20, num_threads=8)

Params:
{'no_components': 72, 'learning_schedule': 'adagrad', 'loss': 'warp', 'learning_rate': 0.06247712728655479, 'item_alpha': 9.490407498434895e-06, 'max_sampled': 11, 'num_epochs': 45, 'rho': 0.9151663627867668, 'epsilon': 9.171970898190447e-06}


Epoch:   0%|          | 0/32 [00:00<?, ?it/s]

Train precision: 0.6410, test precision: 0.2078
Params:
{'no_components': 65, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.022383884192652345, 'item_alpha': 9.557891502289009e-06, 'max_sampled': 13, 'num_epochs': 32, 'rho': 0.8292936663732287, 'epsilon': 3.4102368089271628e-06}


Epoch: 100%|██████████| 32/32 [38:40<00:00, 72.52s/it]


Train precision: 0.6099, test precision: 0.2007
Params:
{'no_components': 72, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.018302438875387395, 'item_alpha': 7.1322848289047725e-06, 'max_sampled': 11, 'num_epochs': 45, 'rho': 0.8338214431327815, 'epsilon': 6.922122716155008e-06}


...

Epoch: 100%|██████████| 49/49 [43:03<00:00, 52.73s/it]


Train precision: 0.6203, test precision: 0.2108
Params:
{'no_components': 83, 'learning_schedule': 'adadelta', 'loss': 'warp', 'learning_rate': 0.04271551982778008, 'item_alpha': 7.013647542683806e-06, 'max_sampled': 13, 'num_epochs': 43, 'rho': 0.8022863069967255, 'epsilon': 9.261414654082415e-06}


Epoch: 100%|██████████| 43/43 [44:41<00:00, 62.37s/it]


Train precision: 0.6593, test precision: 0.1976


(0.21082604,
 {'no_components': 71,
  'learning_schedule': 'adadelta',
  'loss': 'warp',
  'learning_rate': 0.05652168855133591,
  'item_alpha': 2.816672985617472e-06,
  'max_sampled': 9,
  'rho': 0.9369181237831744,
  'epsilon': 2.0854895195545605e-06,
  'num_epochs': 49})

#### WARP with best hyperparameters

In [38]:
model = LightFM(loss='warp', **best_params_so_far)
model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=best_epochs_so_far, num_threads=8)

Epoch: 100%|██████████| 49/49 [43:37<00:00, 53.42s/it]


In [39]:
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8, item_features = id_item_features)

Precision: train 0.62
Precision: test 0.21
AUC: train 0.98
AUC: test 0.93


In [40]:
no_users = len(df['bgg_user_name'].unique())/10
evaluate_diversity_and_coverage(model, int(no_users), dataset, games_df, known, 'data/top10_warp.csv.gz', item_features = id_item_features, N = 10)

100%|██████████| 24757/24757 [02:19<00:00, 177.48it/s]


Diversity: {'category': 0.23476236639180456, 'mechanic': 0.20414428242517269}
Coverage: 3101


### k-OS WARP model

In [24]:
model = LightFM(loss = 'warp-kos')
model.fit(interactions, verbose=True, epochs=20, num_threads=8)

Epoch: 100%|██████████| 20/20 [03:19<00:00,  9.95s/it]


In [25]:
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8)

Precision: train 0.46
Precision: test 0.17
AUC: train 0.92
AUC: test 0.91


In [26]:
id_features_model = LightFM(loss = 'warp-kos')
id_features_model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=20, num_threads=8)

Epoch: 100%|██████████| 20/20 [11:35<00:00, 34.78s/it]


In [27]:
evaluate_model(id_features_model, interactions, test_interactions, k=5,
               num_threads=8, item_features = id_item_features)

Precision: train 0.46
Precision: test 0.17
AUC: train 0.91
AUC: test 0.90


#### k-OS WARP with best hyperparameters (for WARP)

In [44]:
model = LightFM(loss='warp-kos', **best_params_so_far)
model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=best_epochs_so_far, num_threads=8)

Epoch: 100%|██████████| 49/49 [42:32<00:00, 52.09s/it] 


In [45]:
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8, item_features = id_item_features)

Precision: train 0.69
Precision: test 0.19
AUC: train 0.94
AUC: test 0.91


In [46]:
no_users = len(df['bgg_user_name'].unique())/10
evaluate_diversity_and_coverage(model, int(no_users), dataset, games_df, known, 'data/top10_warp-kos.csv.gz', item_features = id_item_features, N = 10)

100%|██████████| 24757/24757 [02:19<00:00, 177.56it/s]


Diversity: {'category': 0.2360823393864554, 'mechanic': 0.20801886645890363}
Coverage: 2724


### BPR model

In [28]:
model = LightFM(loss = 'bpr')
model.fit(interactions, verbose=True, epochs=20, num_threads=8)

Epoch: 100%|██████████| 20/20 [01:40<00:00,  5.01s/it]


In [29]:
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8)

Precision: train 0.45
Precision: test 0.13
AUC: train 0.90
AUC: test 0.87


In [30]:
id_features_model = LightFM(loss = 'bpr')
id_features_model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=20, num_threads=8)

Epoch: 100%|██████████| 20/20 [14:18<00:00, 42.94s/it]


In [31]:
evaluate_model(id_features_model, interactions, test_interactions, k=5,
               num_threads=8, item_features = id_item_features)

Precision: train 0.41
Precision: test 0.12
AUC: train 0.90
AUC: test 0.86


#### BPR with best hyperparameters (for WARP)

In [ ]:
model = LightFM(loss='bpr', no_components = 71)
model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=5, num_threads=8)
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8, item_features = id_item_features)

In [41]:
model = LightFM(loss='bpr', **best_params_so_far)
model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=best_epochs_so_far, num_threads=8)

Epoch: 100%|██████████| 49/49 [1:25:20<00:00, 104.51s/it]


In [42]:
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8, item_features = id_item_features)

Precision: train 0.00
Precision: test 0.00
AUC: train 0.18
AUC: test 0.21


In [43]:
no_users = len(df['bgg_user_name'].unique())/10
evaluate_diversity_and_coverage(model, int(no_users), dataset, games_df, known, 'data/top10_bpr.csv.gz', item_features = id_item_features, N = 10)

100%|██████████| 24757/24757 [02:39<00:00, 155.21it/s]


Diversity: {'category': 0.2438473389921465, 'mechanic': 0.1611749723798841}
Coverage: 1828


#### What causes precision to be 0?

In [47]:
best_params_so_far

{'no_components': 71,
 'learning_schedule': 'adadelta',
 'learning_rate': 0.05652168855133591,
 'item_alpha': 2.816672985617472e-06,
 'max_sampled': 9,
 'rho': 0.9369181237831744,
 'epsilon': 2.0854895195545605e-06}

In [50]:
model = LightFM(loss='bpr', no_components = 71)
model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=5, num_threads=8)
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8, item_features = id_item_features)

Epoch: 100%|██████████| 5/5 [08:26<00:00, 101.29s/it]


Precision: train 0.44
Precision: test 0.12
AUC: train 0.90
AUC: test 0.86


In [51]:
model = LightFM(loss='bpr', no_components = 71, learning_schedule = 'adadelta')
model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=5, num_threads=8)
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8, item_features = id_item_features)

Epoch: 100%|██████████| 5/5 [08:35<00:00, 103.06s/it]


Precision: train 0.57
Precision: test 0.12
AUC: train 0.94
AUC: test 0.87


#### Item_alpha is to big for bpr?

In [52]:
model = LightFM(loss='bpr', no_components = 71, learning_schedule = 'adadelta', item_alpha = 2.816672985617472e-06)
model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=5, num_threads=8)
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8, item_features = id_item_features)

Epoch: 100%|██████████| 5/5 [08:35<00:00, 103.06s/it]


Precision: train 0.15
Precision: test 0.04
AUC: train 0.84
AUC: test 0.83


In [54]:
model = LightFM(loss='bpr', no_components = 71, learning_schedule = 'adadelta', max_sampled = 9)
model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=5, num_threads=8)
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8, item_features = id_item_features)

Epoch: 100%|██████████| 5/5 [08:39<00:00, 103.98s/it]


Precision: train 0.57
Precision: test 0.12
AUC: train 0.94
AUC: test 0.87


In [55]:
model = LightFM(loss='bpr', no_components = 71, learning_schedule = 'adadelta', max_sampled = 9, rho = 0.9369181237831744)
model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=5, num_threads=8)
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8, item_features = id_item_features)

Epoch: 100%|██████████| 5/5 [08:41<00:00, 104.23s/it]


Precision: train 0.55
Precision: test 0.12
AUC: train 0.93
AUC: test 0.87


In [56]:
model = LightFM(loss='bpr', no_components = 71, learning_schedule = 'adadelta', max_sampled = 9, rho = 0.9369181237831744, epsilon = 2.0854895195545605e-06)
model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=5, num_threads=8)
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8, item_features = id_item_features)

Epoch: 100%|██████████| 5/5 [08:40<00:00, 104.18s/it]


Precision: train 0.60
Precision: test 0.12
AUC: train 0.94
AUC: test 0.87


In [57]:
model = LightFM(loss='bpr', no_components = 71, learning_schedule = 'adadelta', max_sampled = 9, rho = 0.9369181237831744, epsilon = 2.0854895195545605e-06)
model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=20, num_threads=8)
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8, item_features = id_item_features)

Epoch: 100%|██████████| 20/20 [34:45<00:00, 104.27s/it]


Precision: train 0.78
Precision: test 0.12
AUC: train 0.96
AUC: test 0.83


In [58]:
model = LightFM(loss='bpr', no_components = 71, learning_schedule = 'adadelta', learning_rate = 0.05652168855133591, max_sampled = 9, rho = 0.9369181237831744, epsilon = 2.0854895195545605e-06)
model.fit(interactions, verbose=True, item_features = id_item_features,
                      epochs=5, num_threads=8)
evaluate_model(model, interactions, test_interactions, k=5, num_threads=8, item_features = id_item_features)

Epoch: 100%|██████████| 5/5 [08:37<00:00, 103.41s/it]


Precision: train 0.59
Precision: test 0.12
AUC: train 0.94
AUC: test 0.87


### Logistic model

In [32]:
model = LightFM(loss = 'logistic')
model.fit(interactions_logistic, verbose=True, epochs=20, num_threads=8)

Epoch: 100%|██████████| 20/20 [01:08<00:00,  3.43s/it]


In [33]:
evaluate_model(model, interactions_logistic, test_interactions_logistic, k=5, num_threads=8)

Precision: train 0.24
Precision: test 0.08
AUC: train 0.80
AUC: test 0.79


In [34]:
id_features_model = LightFM(loss = 'logistic')
id_features_model.fit(interactions_logistic, verbose=True, item_features = id_item_features_logistic,
                      epochs=20, num_threads=8)

Epoch: 100%|██████████| 20/20 [06:49<00:00, 20.47s/it]


In [35]:
evaluate_model(id_features_model, interactions_logistic, test_interactions_logistic, k=5,
               num_threads=8, item_features = id_item_features_logistic)

Precision: train 0.06
Precision: test 0.01
AUC: train 0.67
AUC: test 0.67
